In [45]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader

import matplotlib.pyplot as plt

In [46]:
# device config
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [47]:
device

device(type='cuda')

In [48]:
# hyperparameters
input_size = 784 # 28x28
hidden_size = 100
num_classes = 10
num_epochs = 2
batch_size = 200
learning_rate = 0.001

In [49]:
# DataLoader
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, transform=transforms.ToTensor())

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

data_iter = iter(train_loader)
samples, labels = next(data_iter)



print(samples.shape, labels.shape)

# for i in range(6):
#     plt.subplot(2,3,i+1)
#     plt.imshow(samples[i][0], cmap='gray')
# plt.show()

torch.Size([200, 1, 28, 28]) torch.Size([200])


In [50]:
# Fully connected neural network with one hidden layer

class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.l2 = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        # dont apply softmax here due to cross entropy
        return out
    
model = NeuralNet(input_size, hidden_size, num_classes).to(device)

In [51]:
# loss & Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [52]:
# training loop
n_total_steps = len(train_loader)

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # 100, 1 , 28, 28
        # we need 100, 784
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        
        # forward pass
        outputs = model(images)
        
        # loss
        loss = criterion(outputs, labels)
        
        # backward pass
        loss.backward()
    
        # update
        optimizer.step()
    
        # zero gradients
        optimizer.zero_grad()
        
        if (i+1)%100 == 0:
            print('Epoch {}/{}: step {}/{}, loss = {:.4f}'.format(epoch, num_epochs, i+1, n_total_steps, loss))

Epoch 0/2: step 100/300, loss = 0.4129
Epoch 0/2: step 200/300, loss = 0.3077
Epoch 0/2: step 300/300, loss = 0.3253
Epoch 1/2: step 100/300, loss = 0.2710
Epoch 1/2: step 200/300, loss = 0.3126
Epoch 1/2: step 300/300, loss = 0.2007


In [53]:
# test
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    
    for i, (images, labels) in enumerate(test_loader):
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        
        # value, index
        _, predictions = torch.max(outputs, 1)
        n_correct += labels.shape[0]
        n_correct += (predictions == labels).sum().item()
        
acc = 100 * n_correct/n_correct
print('Accuracy: ', acc)

Accuracy:  100.0
